In [4]:
import findspark
findspark.init('/home/murali/spark-3.3.2-bin-hadoop3')
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('AggFunctions').getOrCreate()

23/04/04 16:41:53 WARN Utils: Your hostname, murali-VirtualBox resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
23/04/04 16:41:53 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/04 16:41:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/04/04 16:41:54 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/04/04 16:41:54 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [ ]:
###  approx_count_distinct()  
  returns the count of distinct items in a group of rows
###  avg()
  returns average of values in a group of rows
###  collect_list()
  returns all values from input column as list with duplicates
###  collect_set()
  returns all values form the input column as a list with out duplicates
###  countDistinct()
  returns numnber of distinct elements in input coulumn
###  Count()
   returns number of elements in a column

In [13]:
from pyspark.sql.functions import approx_count_distinct, avg,collect_list, collect_set,count_distinct, count

simpleData = [
    ('murali','hr',1500),
    ('geetha','it',3000),
    ('chintu','developer',4000),
    ('chinna','developer',3000)
]

schema = ['name','dept','sal']

df = spark.createDataFrame(simpleData,schema)

In [14]:
df.show()

+------+---------+----+
|  name|     dept| sal|
+------+---------+----+
|murali|       hr|1500|
|geetha|       it|3000|
|chintu|developer|4000|
|chinna|developer|3000|
+------+---------+----+



In [23]:
df.select(approx_count_distinct('sal').alias('distinct_sal_cnt')).show()

+----------------+
|distinct_sal_cnt|
+----------------+
|               3|
+----------------+



In [24]:
df.select(avg('sal')).show()

+--------+
|avg(sal)|
+--------+
|  2875.0|
+--------+



In [26]:
df.select(collect_list('sal')).show(truncate=False)

+------------------------+
|collect_list(sal)       |
+------------------------+
|[1500, 3000, 4000, 3000]|
+------------------------+



In [27]:
df.select(collect_set('sal')).show(truncate=False)

+------------------+
|collect_set(sal)  |
+------------------+
|[3000, 4000, 1500]|
+------------------+



In [28]:
df.select(count_distinct('name')).show()

+--------------------+
|count(DISTINCT name)|
+--------------------+
|                   4|
+--------------------+



In [29]:
df.select(count('sal')).show()

+----------+
|count(sal)|
+----------+
|         4|
+----------+



# row_number(), ran(), dense_rank()

we need to partiion the data using window.partitionBy() and for row number and rank, we need order by on partition data using order by clause



In [43]:
emp_path = '/home/murali/Desktop/data_files/emp.csv'

emp = spark.read.csv(emp_path,header=True, inferSchema=True)


dept_path = '/home/murali/Desktop/data_files/dept.csv'
dept = spark.read.csv(dept_path,header=True, inferSchema=True)

joined_df = emp.join(dept, emp.DEPTNO == dept.DEPTNO, 'left_outer').\
select(emp.EMPNO,emp.ENAME,emp.JOB,emp.MGR,emp.HIREDATE,emp.SAL,emp.COMM,emp.DEPTNO,dept.DNAME,dept.LOCATION)
      
#joined_df = emp.join(dept, emp.DEPTNO == dept.DEPTNO, 'left_outer')
joined_df.show()

+-----+------+---------+----+-----------+----+----+------+----------+--------+
|EMPNO| ENAME|      JOB| MGR|   HIREDATE| SAL|COMM|DEPTNO|     DNAME|LOCATION|
+-----+------+---------+----+-----------+----+----+------+----------+--------+
| 7369| SMITH|    CLERK|7902|17-DEC-1980| 800|null|    20|  RESEARCH|  DALLAS|
| 7499| ALLEN| SALESMAN|7698|20-FEB-1981|1600| 300|    30|     SALES| CHICAGO|
| 7521|  WARD| SALESMAN|7698|22-FEB-1981|1250| 500|    30|     SALES| CHICAGO|
| 7566| JONES|  MANAGER|7839|02-APR-1981|2975|null|    20|  RESEARCH|  DALLAS|
| 7654|MARTIN| SALESMAN|7698|28-SEP-1981|1250|1400|    30|     SALES| CHICAGO|
| 7698| BLAKE|  MANAGER|7839|01-MAY-1981|2850|null|    30|     SALES| CHICAGO|
| 7782| CLARK|  MANAGER|7839|09-JUN-1981|2450|null|    10|ACCOUNTING|NEW YORK|
| 7788| SCOTT|  ANALYST|7566|19-APR-1987|3000|null|    20|  RESEARCH|  DALLAS|
| 7839|  KING|PRESIDENT|null|17-NOV-1981|5000|null|    10|ACCOUNTING|NEW YORK|
| 7844|TURNER| SALESMAN|7698|08-SEP-1981|1500|   0| 

In [51]:
# row_number 
joined_df.sort('sal').show()

+-----+------+---------+----+-----------+----+----+------+----------+--------+
|EMPNO| ENAME|      JOB| MGR|   HIREDATE| SAL|COMM|DEPTNO|     DNAME|LOCATION|
+-----+------+---------+----+-----------+----+----+------+----------+--------+
| 7369| SMITH|    CLERK|7902|17-DEC-1980| 800|null|    20|  RESEARCH|  DALLAS|
| 7900| JAMES|    CLERK|7698|03-DEC-1981| 950|null|    30|     SALES| CHICAGO|
| 7876| ADAMS|    CLERK|7788|23-MAY-1987|1100|null|    20|  RESEARCH|  DALLAS|
| 7654|MARTIN| SALESMAN|7698|28-SEP-1981|1250|1400|    30|     SALES| CHICAGO|
| 7521|  WARD| SALESMAN|7698|22-FEB-1981|1250| 500|    30|     SALES| CHICAGO|
| 7934|MILLER|    CLERK|7782|23-JAN-1982|1300|null|    10|ACCOUNTING|NEW YORK|
| 7844|TURNER| SALESMAN|7698|08-SEP-1981|1500|   0|    30|     SALES| CHICAGO|
| 7499| ALLEN| SALESMAN|7698|20-FEB-1981|1600| 300|    30|     SALES| CHICAGO|
| 7782| CLARK|  MANAGER|7839|09-JUN-1981|2450|null|    10|ACCOUNTING|NEW YORK|
| 7698| BLAKE|  MANAGER|7839|01-MAY-1981|2850|null| 

In [81]:
from pyspark.sql.functions import rank,dense_rank,row_number, desc
from pyspark.sql.window import Window
from pyspark.sql.functions import monotonically_increasing_id

df_seqNo = joined_df.withColumn("seq_num", monotonically_increasing_id()+1)
df_seqNo.show()

+-----+------+---------+----+-----------+----+----+------+----------+--------+-------+
|EMPNO| ENAME|      JOB| MGR|   HIREDATE| SAL|COMM|DEPTNO|     DNAME|LOCATION|seq_num|
+-----+------+---------+----+-----------+----+----+------+----------+--------+-------+
| 7369| SMITH|    CLERK|7902|17-DEC-1980| 800|null|    20|  RESEARCH|  DALLAS|      1|
| 7499| ALLEN| SALESMAN|7698|20-FEB-1981|1600| 300|    30|     SALES| CHICAGO|      2|
| 7521|  WARD| SALESMAN|7698|22-FEB-1981|1250| 500|    30|     SALES| CHICAGO|      3|
| 7566| JONES|  MANAGER|7839|02-APR-1981|2975|null|    20|  RESEARCH|  DALLAS|      4|
| 7654|MARTIN| SALESMAN|7698|28-SEP-1981|1250|1400|    30|     SALES| CHICAGO|      5|
| 7698| BLAKE|  MANAGER|7839|01-MAY-1981|2850|null|    30|     SALES| CHICAGO|      6|
| 7782| CLARK|  MANAGER|7839|09-JUN-1981|2450|null|    10|ACCOUNTING|NEW YORK|      7|
| 7788| SCOTT|  ANALYST|7566|19-APR-1987|3000|null|    20|  RESEARCH|  DALLAS|      8|
| 7839|  KING|PRESIDENT|null|17-NOV-1981|50

In [89]:
window_spec = Window.partitionBy('DEPTNO').orderBy(desc('SAL'))
df_seqNo.withColumn('rank_num', rank().over(window_spec)).show()

+-----+------+---------+----+-----------+----+----+------+----------+--------+-------+--------+
|EMPNO| ENAME|      JOB| MGR|   HIREDATE| SAL|COMM|DEPTNO|     DNAME|LOCATION|seq_num|rank_num|
+-----+------+---------+----+-----------+----+----+------+----------+--------+-------+--------+
| 7839|  KING|PRESIDENT|null|17-NOV-1981|5000|null|    10|ACCOUNTING|NEW YORK|      9|       1|
| 7782| CLARK|  MANAGER|7839|09-JUN-1981|2450|null|    10|ACCOUNTING|NEW YORK|      7|       2|
| 7934|MILLER|    CLERK|7782|23-JAN-1982|1300|null|    10|ACCOUNTING|NEW YORK|     14|       3|
| 7788| SCOTT|  ANALYST|7566|19-APR-1987|3000|null|    20|  RESEARCH|  DALLAS|      8|       1|
| 7902|  FORD|  ANALYST|7566|03-DEC-1981|3000|null|    20|  RESEARCH|  DALLAS|     13|       1|
| 7566| JONES|  MANAGER|7839|02-APR-1981|2975|null|    20|  RESEARCH|  DALLAS|      4|       3|
| 7876| ADAMS|    CLERK|7788|23-MAY-1987|1100|null|    20|  RESEARCH|  DALLAS|     11|       4|
| 7369| SMITH|    CLERK|7902|17-DEC-1980

In [88]:
#row_number()
window_spec = Window.partitionBy('DEPTNO').orderBy('SAL')
df_seqNo.withColumn('row_num', row_number().over(window_spec)).show()

+-----+------+---------+----+-----------+----+----+------+----------+--------+-------+-------+
|EMPNO| ENAME|      JOB| MGR|   HIREDATE| SAL|COMM|DEPTNO|     DNAME|LOCATION|seq_num|row_num|
+-----+------+---------+----+-----------+----+----+------+----------+--------+-------+-------+
| 7934|MILLER|    CLERK|7782|23-JAN-1982|1300|null|    10|ACCOUNTING|NEW YORK|     14|      1|
| 7782| CLARK|  MANAGER|7839|09-JUN-1981|2450|null|    10|ACCOUNTING|NEW YORK|      7|      2|
| 7839|  KING|PRESIDENT|null|17-NOV-1981|5000|null|    10|ACCOUNTING|NEW YORK|      9|      3|
| 7369| SMITH|    CLERK|7902|17-DEC-1980| 800|null|    20|  RESEARCH|  DALLAS|      1|      1|
| 7876| ADAMS|    CLERK|7788|23-MAY-1987|1100|null|    20|  RESEARCH|  DALLAS|     11|      2|
| 7566| JONES|  MANAGER|7839|02-APR-1981|2975|null|    20|  RESEARCH|  DALLAS|      4|      3|
| 7788| SCOTT|  ANALYST|7566|19-APR-1987|3000|null|    20|  RESEARCH|  DALLAS|      8|      4|
| 7902|  FORD|  ANALYST|7566|03-DEC-1981|3000|null

In [87]:
#dennce_rank
window_spec = Window.partitionBy('DEPTNO').orderBy(desc('SAL'))
df_seqNo.withColumn('dense_rank_num', dense_rank().over(window_spec)).show()

+-----+------+---------+----+-----------+----+----+------+----------+--------+-------+--------+
|EMPNO| ENAME|      JOB| MGR|   HIREDATE| SAL|COMM|DEPTNO|     DNAME|LOCATION|seq_num|rank_num|
+-----+------+---------+----+-----------+----+----+------+----------+--------+-------+--------+
| 7839|  KING|PRESIDENT|null|17-NOV-1981|5000|null|    10|ACCOUNTING|NEW YORK|      9|       1|
| 7782| CLARK|  MANAGER|7839|09-JUN-1981|2450|null|    10|ACCOUNTING|NEW YORK|      7|       2|
| 7934|MILLER|    CLERK|7782|23-JAN-1982|1300|null|    10|ACCOUNTING|NEW YORK|     14|       3|
| 7788| SCOTT|  ANALYST|7566|19-APR-1987|3000|null|    20|  RESEARCH|  DALLAS|      8|       1|
| 7902|  FORD|  ANALYST|7566|03-DEC-1981|3000|null|    20|  RESEARCH|  DALLAS|     13|       1|
| 7566| JONES|  MANAGER|7839|02-APR-1981|2975|null|    20|  RESEARCH|  DALLAS|      4|       2|
| 7876| ADAMS|    CLERK|7788|23-MAY-1987|1100|null|    20|  RESEARCH|  DALLAS|     11|       3|
| 7369| SMITH|    CLERK|7902|17-DEC-1980